# GloVeの単語ベクトルによる類似語検索

In [62]:
from gensim.models import KeyedVectors

#glove_vector_file = "../glove/gensim_glove_vectors.txt"
glove_vector_file = "../elasticsearch/es_gensim_glove_vectors.txt"
word = "トラック"
top_k = 30

glove_vectors = KeyedVectors.load_word2vec_format(glove_vector_file, binary=False)
glove_vectors.most_similar(word, [], top_k)

[('大型', 0.812087893486023),
 ('トレーラー', 0.8037623167037964),
 ('t', 0.8022152781486511),
 ('停車', 0.7754109501838684),
 ('ダンプ', 0.7720902562141418),
 ('荷台', 0.765929102897644),
 ('積む', 0.7538949847221375),
 ('走行', 0.753769040107727),
 ('ダンプトラック', 0.7408422827720642),
 ('走行中', 0.7344934940338135),
 ('追突', 0.7296924591064453),
 ('ダンプカー', 0.7262539267539978),
 ('前方', 0.7261230945587158),
 ('くる', 0.7258666753768921),
 ('10', 0.7164998054504395),
 ('積載', 0.7143024206161499),
 ('国道', 0.7056641578674316),
 ('後方', 0.6954992413520813),
 ('運転', 0.6948610544204712),
 ('トントラック', 0.6948599815368652),
 ('積み込む', 0.6935635805130005),
 ('自動車', 0.6884791851043701),
 ('車両', 0.6840767860412598),
 ('4', 0.6738653182983398),
 ('後部', 0.6726698279380798),
 ('貨物自動車', 0.6653414964675903),
 ('運転手', 0.6386405825614929),
 ('衝突', 0.6329050660133362),
 ('乗用車', 0.6329023838043213),
 ('高速道路', 0.6220003366470337)]

In [50]:
import numpy as np

def cos_similarity(x, y, eps=1e-8):
    '''コサイン類似度の算出
    :param x: ベクトル
    :param y: ベクトル
    :param eps: ”0割り”防止のための微小値
    :return:
    '''
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)

def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
    '''類似単語の検索
    :param query: クエリ（テキスト）
    :param word_to_id: 単語から単語IDへのディクショナリ
    :param id_to_word: 単語IDから単語へのディクショナリ
    :param word_matrix: 単語ベクトルをまとめた行列。各行に対応する単語のベクトル
が格納されていることを想定する
    :param top: 上位何位まで表示するか
    '''
    if query not in word_to_id:
        print('%s is not found' % query)
        return

    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

    vocab_size = len(id_to_word)

    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

    count = 0
    for i in (-1 * similarity).argsort():
        print("i:{}".format(i))
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

In [48]:
import pandas as pd
import pickle
import numpy as np

target_data = 1100
top_k = 10

df = pd.read_csv('wakati_category_all.csv')
with open('scdv.pickle', 'rb') as f:
    scdv = pickle.load(f)

sentences_len = len(df)    

sims = np.array([cos_similarity(scdv[target_data], scdv[i]) for i in range(sentences_len)])

topk_index = np.argsort(-sims)[:top_k]

print("{}, {}".format(df['業種(大分類)'][target_data], df['文章'][target_data]))
print("================")
for i in range(top_k):
    index = topk_index[i]
    print("{}, {}, {}".format(sims[index], df['業種(大分類)'][index], df['文章'][index]))


農林業, 機械集材装置により丸太を運搬し土場へ降ろす作業を単独で行っていて、丸太の下敷きとなった。
0.9999999403953552, 農林業, 機械集材装置により丸太を運搬し土場へ降ろす作業を単独で行っていて、丸太の下敷きとなった。
0.7055249214172363, 製造業, 製材工場の皮剥機での丸太の加工中、台車と操作室との間にある配線カバーが外れているのに気付いた操作者が、修理しようとして一旦皮剥機の稼働を停止して丸太を卸したとき、台車上の木材の向こう側にいた被災者に丸太が激突した。丸太は直径６０ｃｍ、長さ６．２ｍ、重量が約１．１ｔの米松である。
0.7026406526565552, 運輸交通業, 倉庫敷地内において、作業者２名が（丸太の積み込み、運搬等を行う）専用の荷役機械を用いて、２０ｔトレーラーに、約４ｍの丸太を縦三列に積み込む作業中、荷台近くにいたトレーラの運転手が、落ちてきた１本の丸太に激突された。
0.690472424030304, 農林業, 山土場に仮集積していた木(15本)を、機械集材装置を用いて、土場に運搬する作業中、吊り上げた丸太の内の1本が落下し、荷掛け作業等を行っていた者に当たった。
0.6887274980545044, 農林業, 機械集材装置により伐木を土場まで運搬する作業中、伐木が被災者に激突した。
0.6844440698623657, 製造業, ２基で１対となった機械を工場に設置するため鋼製のパレットに乗せて搬入し、同機械をパレットから降ろす作業中、同機械の下部アームに乗り２基を結んでいた番線を解いたところ、同機械が倒れ下敷きになった。
0.6723582744598389, 農林業, 機械集材装置を用いて玉切り材の荷卸し作業中、転倒した。
0.6654088497161865, 製造業, 自動送材装置付ダブルソーにより丸太材を角材に製材する作業で、丸太が送り出されて製材が始まったので次の丸太を準備しようとしたところ、その丸太が停止位置を越え自動送材装置のそばまで転がって、すでに1回目の製材が終わり戻ってくる加工材とその丸太が引っかかり、自動送材装置で制御盤側に押し出された丸太に激突された。
0.6615996956825256, 製造業, 製材工場において、丸太１７本（計６．０３ｔ）を積んだトラック（最大積載量